In [ ]:
from user_constants import *
import tpu_simulation_utilities
import tensorflow.compat.v1 as tf
import numpy as np
import os
import io
import sys
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
from skimage import transform
import ipywidgets as widgets
from osgeo import gdal
from typing import Any, Callable, Iterable, Sequence, List, Dict, Mapping, MutableMapping, MutableSequence, Optional, Text, Tuple, Union

if PUBLIC_COLAB:
  # Authenticate to access Google Cloud Storage.
  from google.colab import auth  # pylint: disable=g-import-not-at-top
  auth.authenticate_user()

def parabola(y, x, L, nx, ny, dem):
    a = 10
    h0 = 5
    B = tf.math.sqrt(2 * _G * h0) / (2*a)
    x1_0 =  (1/2) - a + (L / 2)
    x2_0 =  (1/2) + a + (L / 2)
    z = - h0 * (tf.math.square( (1/a)* (y-(L/2)) - (B/tf.math.sqrt(2 * _G * h0))) - 1) + SAINT_VENANT_EPS
    z = tf.where(y < x1_0, tf.ones_like(z)*SAINT_VENANT_EPS, z)
    z = tf.where(y > x2_0, tf.ones_like(z)*SAINT_VENANT_EPS, z)
    z = tf.expand_dims(z,axis=0)
    z = tf.repeat(z, nx,axis = 0) 
    return z

def run_simulation(
    dem_tiff_filename: Text, resolution: int, num_secs: float,
    num_secs_per_cycle: float, dt: float, cx: int, cy: int,
    run_dir: Text, start_time_secs: float = 0) -> None:
  """Runs the simulation."""
  # Necessary Component of Running Simulation, no editing here.
  slope = 1e-4
  input_file_format = ('{}' + f'-{start_time_secs}.np' if start_time_secs > 0
                       else None)

  unpadded_dem,geoinfo = tpu_simulation_utilities.load_dem_from_tiff_file(dem_tiff_filename,resolution)
  unpadded_river_mask = np.ones_like(unpadded_dem, dtype=bool)
  unpadded_manning_matrix = tpu_simulation_utilities.get_manning_matrix_from_river_mask(
      unpadded_river_mask, MANNING_COEFF_RIVER, MANNING_COEFF_FLOODPLAIN)
  params = tpu_simulation_utilities.get_sv_params(unpadded_dem.shape, resolution, num_secs,
                         num_secs_per_cycle, cx, cy, dt)
  print('params:', params)
  
  # SETS THE BOUNDARY CONDITIONS FOR HEIGHT, FLUX IN Y, FLUX IN X
  h_bcs = [
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.LEFT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0*3940,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.RIGHT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)
  ]

  q_y_bcs = [
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.LEFT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.DirichletBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.RIGHT,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)
  ]

  q_x_bcs = [
     tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.TOP,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix),
      tpu_simulation_utilities.NeumannBoundary(
          boundary_side=tpu_simulation_utilities.BoundarySide.BOTTOM,
          fraction_start=0,
          fraction_end=1.0,
          left_padding=params.left_padding,
          top_padding=params.top_padding,
          slope=slope,
          value=0.0,
          unpadded_dem=unpadded_dem,
          unpadded_manning_matrix=unpadded_manning_matrix)]

  # NO NEED TO EDIT THE FOLLOWING
  init_files_manager = tpu_simulation_utilities.InitFilesManager(
      params, tpu_simulation_utilities.three_d_subgrid_of_2d_grid, run_dir)

  # LAST TWO ARGUMENTS SET INITIAL CONDITIONS IN HEIGHT AND FLUX IN Y DIRECTION
  sim_builder = tpu_simulation_utilities.get_sim_builder(
      params, unpadded_dem, unpadded_manning_matrix,h_bcs,q_x_bcs,q_y_bcs,
      init_files_manager, input_file_format, start_time_secs, 
      parabola, tpu_simulation_utilities.constant_height_zeros)

  def session_runner_builder(sim):
    return tpu_simulation_utilities.get_session_runner_builder(params, init_files_manager, run_dir, sim)

  manager = tpu_simulation_utilities.TPUSimulationManager(params, sim_builder, session_runner_builder)
  manager.run_simulation(start_time_secs)

In [ ]:
# SIMULATION PARAMETERS - NEED TO BE SET
resolution, dt, cx = 4, 100000, 5e-3, 1
start_time_secs, num_secs, num_secs_per_cycle = 0, 400.0,10.0
# SIMULATION DEM FILE NAME
dem_bucket_filename = f'gs://{BUCKET}/parabola.tif'

# NO NEED TO EDIT THIS COMPONENT
dem_tiff_filename = tpu_simulation_utilities.get_dem_tiff_filename(dem_bucket_filename,resolution)
run_dir = f'gs://{BUCKET}'
if PUBLIC_COLAB:
  cy = 8
  os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
  TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

In [ ]:
# RUN SIMULATION ON TPU
if PUBLIC_COLAB:
  g = tf.Graph()
  run_meta = tf.RunMetadata()
  with g.as_default():
    run_simulation(dem_tiff_filename, resolution, num_secs,
                  num_secs_per_cycle, dt, cx, cy, run_dir,
                  start_time_secs)

In [ ]:
# Load the DEM and the heightmaps.
dem, geoinfo = tpu_simulation_utilities.load_dem_from_tiff_file(dem_tiff_filename,resolution)
dem_shape = dem.shape
heightmaps = {}
t_range = np.arange(start_time_secs,num_secs + num_secs_per_cycle,num_secs_per_cycle)
for t in t_range:
  t  = round(t,3)
  filename = os.path.join(run_dir, f'h-{t}.np')
  print(f'Loading heightmap: h-{t}.np')
  with tf.io.gfile.GFile(filename, 'rb') as f:
    heightmaps[t] = np.load(f)
  
resize_factor = 1
resize_shape = [dem_shape[0] // resize_factor, dem_shape[1] // resize_factor]
scaled_dem = transform.resize(dem, resize_shape)
scaled_heightmaps = {}

for ts, hmap in heightmaps.items():
  scaled_heightmaps[ts] = transform.resize(hmap, resize_shape)

In [ ]:
def parabola_verification_no_friction(t, y, a, h0, L):
    scaling = np.sqrt(2*_G*h0)
    x1 = 0.5 * np.cos(scaling * t / a) - a + (L/2)
    x2 = 0.5 * np.cos(scaling * t / a) + a + (L/2)
    h = -h0 * (np.square((1/a)*(y-(L/2)) - (1/(2*a)) * np.cos(scaling *t / a))-1)
    h = np.where( np.logical_or(y < x1,y > x2), np.zeros_like(h), h)
    return h

In [ ]:
%matplotlib inline
def plot_parabola(time_secs,ax=None):
  if (ax == None):
    plt.figure(figsize=(8, 8), dpi=100)
    ax = plt.gca()
  t = time_secs
  row = 10
  ax.plot(np.linspace(0,40,401), scaled_dem[row,:]+scaled_heightmaps[t][row,:], label='Computed Solution',color='#7570b3')
  x = np.linspace(0,40,401)
  ax.fill_between(x,np.ones_like(scaled_dem[row,:])*-8, scaled_dem[row,:],color='gray',alpha=0.25,label='Topography')
  ax.plot(x, scaled_dem[row,:]+parabola_verification_no_friction(t,x, 10,5,40)+1e-4, linestyle='dashed',label='Analytical Solution',color='#fc8d62')
  ax.set_title('Parabolic Bowl at t = ' + str(t) + ' seconds')
  ax.set_ylim([-7,10])
  ax.set_xlim([0,40])
  ax.legend()

def plot_error(ax):
  linf ={}
  l2norm = {}
  row = 100
  x = np.linspace(0,40,401)
  for ts, hmap in scaled_heightmaps.items():
    linf[ts] = (np.max(abs(hmap[row,1:-1]-parabola_verification_no_friction(ts,x, 10,5,40)[1:-1]-1e-4)))/ (np.max(abs(parabola_verification_no_friction(ts,x, 10,5,40)+1e-4)))
    l2norm[ts] = np.sqrt(np.sum(np.square(hmap[row,1:-1]-parabola_verification_no_friction(ts,x, 10,5,40)[1:-1]-1e-4))) / np.sqrt(np.sum(np.square(parabola_verification_no_friction(ts,x, 10,5,40)+1e-4)))
  fig = plt.figure(figsize=(6,4))
  ax.plot(np.array([*linf]), [*linf.values()], label= '$L_\infty$',color='#1f78b4')
  ax.plot(np.array([*l2norm]), [*l2norm.values()],label='$L_2$',color='#33a02c')
  ax.set_yscale('log')
  ax.set_xlabel('Time [s]')
  ax.set_title('Relative Error in Norm of Parabolic Bowl Solution')
  ax.legend(fontsize='x-large')
if PUBLIC_COLAB:
  slider = widgets.FloatSlider(
      value=num_secs,
      min=start_time_secs, max=num_secs,
      step=num_secs_per_cycle, readout_format='.4f')
  out = widgets.interactive_output(
      lambda time_secs: plot_parabola(time_secs), {'time_secs': slider})
  display(widgets.VBox([slider, out]))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15,3),dpi=100)
plot_parabola(3.5,ax[0])
plot_parabola(12.5,ax[1])
plot_error(ax[2])